### Downloading the Dogs vs Cats dataset

In [ ]:
# !pip install tensorflow-gpu==2.0

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2023-12-03 06:32:50--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.146.207, 209.85.147.207, 142.250.125.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.146.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   164MB/s    in 0.4s    

2023-12-03 06:32:50 (164 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Stage 2: Dataset preprocessing

### Import dependencies

In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.14.0'

### Unzipping the Dogs vs Cats dataset

In [ ]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [ ]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [ ]:
zip_object.extractall("./")

In [ ]:
zip_object.close()

### Seting up dataset paths

In [ ]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [ ]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [ ]:
IMG_SHAPE = (128, 128, 3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Conv1[

### Freezing the base model

In [ ]:
base_model.trainable = False

### Defining the custom head for our network

In [ ]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [ ]:
dense=tf.keras.layers.Dense(units=1000,activation='relu')(global_average_layer)
dropout=tf.keras.layers.Dropout(0.3)(dense)

In [ ]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(dropout)

### Defining the model

In [ ]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 64, 64, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 64, 64, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 64, 64, 32)           0         ['bn_Conv1[0][0]']        

### Compiling the model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

 MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [ ]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [ ]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=40, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [ ]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=40, class_mode="binary")

Found 2000 images belonging to 2 classes.


### Training the model

In [ ]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

Epoch 1/10


<ipython-input-25-9b4cb72bc593>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)


50/50 [==============================] - 18s 129ms/step - loss: 0.2894 - accuracy: 0.9215 - val_loss: 0.3471 - val_accuracy: 0.8940
Epoch 2/10
50/50 [==============================] - 6s 128ms/step - loss: 0.1011 - accuracy: 0.9635 - val_loss: 0.0907 - val_accuracy: 0.9690
Epoch 3/10
50/50 [==============================] - 5s 97ms/step - loss: 0.0709 - accuracy: 0.9765 - val_loss: 0.1781 - val_accuracy: 0.9480
Epoch 4/10
50/50 [==============================] - 5s 95ms/step - loss: 0.0397 - accuracy: 0.9860 - val_loss: 0.0944 - val_accuracy: 0.9750
Epoch 5/10
50/50 [==============================] - 6s 123ms/step - loss: 0.0383 - accuracy: 0.9855 - val_loss: 0.1293 - val_accuracy: 0.9670
Epoch 6/10
50/50 [==============================] - 5s 97ms/step - loss: 0.0276 - accuracy: 0.9885 - val_loss: 0.1880 - val_accuracy: 0.9530
Epoch 7/10
50/50 [==============================] - 6s 118ms/step - loss: 0.0211 - accuracy: 0.9920 - val_loss: 0.1272 - val_accuracy: 0.9700
Epoch 8/10
50/50 [=

model.fit(train_generator, callbacks=[tensorboard_callback], epochs=10,validation_data=valid_generator)


### Transfer learning model evaluation

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

<ipython-input-26-a86cc1f7f1e2>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9739999771118164


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model.

### Un-freeze a few top layers from the model

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [ ]:
fine_tune_at = 90

In [ ]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.00015),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [ ]:
model.fit_generator(train_generator,
                    epochs=10,
                    validation_data=valid_generator)

Epoch 1/10


<ipython-input-30-52777b91ac85>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


50/50 [==============================] - 10s 143ms/step - loss: 2.6273e-04 - accuracy: 1.0000 - val_loss: 0.1405 - val_accuracy: 0.9720
Epoch 2/10
50/50 [==============================] - 5s 97ms/step - loss: 8.3963e-05 - accuracy: 1.0000 - val_loss: 0.1451 - val_accuracy: 0.9720
Epoch 3/10
50/50 [==============================] - 5s 96ms/step - loss: 1.6556e-04 - accuracy: 1.0000 - val_loss: 0.1448 - val_accuracy: 0.9720
Epoch 4/10
50/50 [==============================] - 6s 118ms/step - loss: 6.4166e-05 - accuracy: 1.0000 - val_loss: 0.1504 - val_accuracy: 0.9720
Epoch 5/10
50/50 [==============================] - 5s 98ms/step - loss: 5.6743e-05 - accuracy: 1.0000 - val_loss: 0.1488 - val_accuracy: 0.9710
Epoch 6/10
50/50 [==============================] - 6s 114ms/step - loss: 4.6960e-05 - accuracy: 1.0000 - val_loss: 0.1566 - val_accuracy: 0.9730
Epoch 7/10
50/50 [==============================] - 5s 98ms/step - loss: 2.7763e-05 - accuracy: 1.0000 - val_loss: 0.1571 - val_accuracy:

### Evaluating the fine tuned model

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

<ipython-input-31-a86cc1f7f1e2>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


In [ ]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.972000002861023
